# Импорты библиотек

In [3]:
import numpy
from plotly import express as px
import time

# Элементарная нижняя треугольная матрица

Элементарная нижняя треугольная матрица для матрицы $A$ размера $n \times m$ $-$ это матрица размера $n \times n$ вида:</br>
$
L_i = \left( \begin{array}{cccccc}
    1 & \cdots & 0 & 0 & \cdots & 0 \\
    \vdots & \ddots & \vdots & \vdots & \ddots & \vdots \\
    0 & \cdots & \displaystyle \frac{1}{a_{ii}} & 0 & \cdots & 0 \\
    \\
    0 & \cdots & \displaystyle -\frac{a_{i + 1, i}}{a_{ii}} & 1 & \cdots & 0 \\
    \vdots & \ddots & \vdots & \vdots & \ddots & \vdots \\
    0 & \cdots & \displaystyle -\frac{a_{ni}}{a_{ii}} & 0 & \cdots & 1
\end{array} \right), i = \overline{1, n} \\
$</br>, где $a_{ji}, j = \overline{i, n} -$ это элементы матрицы $A$ на позиции $ji$.

In [4]:
def elementary_lower_triangular_matrix(i: int, A: numpy.array) -> numpy.array:
    n = A.shape[0]
    L = numpy.eye(n)

    L[i, i] = 1.0 / A[i, i]
    for j in range(i + 1, n):
        L[j, i] = -A[j, i] * L[i, i]

    return L

# Элементарная матрица перестановок

Элементарная матрица перестановок $P_{ij}$ $-$ матрица, полученная из единичной матрицы перестановкой $i$-ой и $j$-ой строк.

In [5]:
def elementary_permutation_matrix(i: int, j: int, n: int) -> numpy.array:
    P = numpy.eye(n)
    if i != j:
        P[[i, j]] = P[[j, i]]
    return P

# Метод Гаусса с выбором максимального элемента по матрице

Суть данного метода заключается в том, чтобы при $k$-ом шаге прямого хода метода Гаусса в качестве элемента $a_{kk}$ стоял максимальный элемент по модулю в матрице $A_k^{(k - 1)}$, который является подматрицей $A^{(k - 1)}$ с элементами, индексы строк и столбцов которых больше или равен $k$:</br>
$
A^{(k - 1)} = (a^{(k - 1)})_{i, j}^n \\
A_k^{(k - 1)} = (a^{(k - 1)})_{i, j}^n, \forall i, j \geq k \\
$, где $A^{(k - 1)}$ $-$ изменённая матрица $A$ на $(k - 1)$-ом шаге прямого хода метода Гаусса.

У нас имеется СЛАУ вида:</br>
$Ax = b \\$
, где $A$ $-$ матрица $n \times n$, $b$ $-$ вектор из $\mathbb{R}^n$. </br>
На первом шаге мы должны найти элемент $a_{ij} = \max |A|$, затем переместить его на место $a_{11}$. Для этого сначала нужно поменять местами 1-ю и $i$-ю строки, точно также местами поменяются элементы вектора $b$. Это эквивалентно тому, чтобы домножить на матрицу $A$ и вектор $b$ элементарную матрицу перестановок $P_{1i}$ слева:</br>
$P_{1i} Ax = P_{1i} b \\$
Далее необходимо поменять местами 1-й и $j$-й столбцы, точно также местами поменяются элементы вектора $x$. Это эквивалентно тому, чтобы домножить на матрицу $A$ элементарную матрицу перестановок $P_{1j}$ справа и домножить на вектор $x$ обратную элементарную матрицу перестановок $P_{1j}^{-1}$ слева:</br>
$P_{1i} A P_{1j} (P_{1j}^{-1} x) = P_{1i} b \\$
Таким образом совершается эквивалентное преобразование исходной системы, приводящее к тому, чтобы на позиции $a_{11}$ стоял максимальный элемент матрицы $A$ по модулю. Далее в прямом ходе метода Гаусса совершается эквивалентное преобразование, которое заключается в том, чтобы $a_{11}$ стал 1, а все элементы ниже него занулились. Для этого 1-я строка умножается на $\displaystyle \frac{1}{a_{11}}$, а $i$-я строка умножается на $\displaystyle -\frac{a_{11}}{a_{i1}}$ и суммируется с 1-й строкой. Это эквивалентно тому, чтобы домножить на матрицу $P_{1i} A P_{1j}$ и на вектор $P_{1i} b$ её элементарную нижнюю треугольную матрицу $L_1$ слева:</br>
$L_1 P_{1i} A P_{1j} (P_{1j}^{-1} x) = L_1 P_{1i} b \\$
Таким образом получим СЛАУ, эквивалентная исходной, вида:</br>
$A^{(1)} x^{(1)} = b^{(1)} \\$
, где $A^{(1)} = L_1 P_{1i} A P_{1j}$, $x^{(1)} = P_{1j}^{-1} x$, $b^{(1)} = L_1 P_{1i} b$

На $(k - 1)$-ом шаге метода у нас будет СЛАУ:</br>
$A^{(k - 1)} x^{(k - 1)} = b^{(k - 1)} \\$
Действуя аналогично, приходим к новой СЛАУ вида:</br>
$L_k P_{ki} A^{(k - 1)} P_{kj} (P_{kj}^{-1} x^{(k - 1)}) = L_k P_{ki} b^{(k - 1)} \\$

Совершив таких $n$ шагов, получим итоговую СЛАУ:</br>
$
L_n P_{ni} A^{(n - 1)} P_{nj} (P_{nj}^{-1} x^{(n - 1)}) = L_n P_{ni} b^{(n - 1)} \\
A^{(n)} x^{(n)} = b^{(n)} \\
$

Этим завершается прямой ход метода Гаусса. На обратном ходе Гаусса совершается вычисление элементов вектора $x^{(n)}$:</br>
$\displaystyle
x_n^{(n)} = \frac{b_n^{(n)}}{a_{nn}^{(n)}} \\
$
</br>
$\displaystyle
x_i^{(n)} = \frac{1}{a_{ii}^{(n)}} \left( b_i^{(n)} - \sum_{j = i + 1}^n a_{ij} x_j^{(n)} \right), i = \overline{n - 1, 1} \\
$
Таким образом получим значения для $x^{(n)}$, вспомним, что:</br>
$x^{(n)} = P_{nj_{n}}^{-1} x^{(n - 1)} = P_{nj_{n}}^{-1} P_{n - 1, j_{n - 1}}^{-1} x^{(n - 2)} = \ldots = P_{nj_{n}}^{-1} P_{n - 1, j_{n - 1}}^{-1} \ldots P_{1j_{1}}^{-1} x \\$
Тогда $x$ можно найти по формуле:</br>
$x = P_{1j_{1}} P_{2j_{2}} \ldots P_{nj_{n}} x^{(n)}$

In [6]:
def gaussian_method_with_maximum_element_by_matrix(A: numpy.array, b: numpy.array) -> numpy.array:
    n = A.shape[0]
    P_x = numpy.eye(n)

    # Прямой ход
    for k in range(n):
        A_k = numpy.abs(A[k:, k:])
        max_i, max_j = numpy.unravel_index(numpy.argmax(A_k), A_k.shape)

        max_i += k
        max_j += k

        if (A[max_i, max_j] == 0):
            raise ValueError("Система не имеет единственного решения")

        A[[k, max_i]] = A[[max_i, k]]
        b[k], b[max_i] = b[max_i], b[k]

        A[:, [k, max_j]] = A[:, [max_j, k]]
        P = elementary_permutation_matrix(k, max_j, n)
        P_x = P_x @ P

        L_k = elementary_lower_triangular_matrix(k, A)

        A = L_k @ A
        b = L_k @ b

    # Обратный ход
    y = numpy.zeros(n)
    y[n - 1] = b[n - 1] / A[n - 1, n - 1]
    for k in range(n - 2, -1, -1):
        y[k] = (b[k] - (A[k, k + 1:] @ y[k + 1:])) / A[k, k]

    # Восстановление x
    x = P_x @ y
    return x

# Метод Гаусса с выбором максимального элемента по столбцу

Данный метод похож на предыдущий с одним отличием, что поиск максимального элемента происходит не по всей матрице, а лишь по столбцу. Таким образом на каждом $k$-ом шаге преобразованное СЛАУ выглядит так:</br>
$L_k P_{ki} A^{(k - 1)} x = L_k P_{ki} b^{(k - 1)} \\$
Тогда итоговое СЛАУ имеет вид:</br>
$
L_n P_{ni} A^{(n - 1)} x = L_n P_{ni} b^{(n - 1)} \\
A^{(n)} x = b^{(n)} \\
$
В связи с этим после обратного хода получается решение исходной системы, так как никаких преобразований вектора $x$ не происходило

In [7]:
def gaussian_method_with_maximum_element_by_column(A: numpy.array, b: numpy.array) -> numpy.array:
    n = A.shape[0]

    # Прямой ход
    for k in range(n):
        A_k = numpy.abs(A[k:, k])
        max_i = numpy.unravel_index(numpy.argmax(A_k), A_k.shape)[0]
        max_i += k

        if (A[max_i, k] == 0):
            raise ValueError("Система не имеет единственного решения")

        A[[k, max_i]] = A[[max_i, k]]
        b[k], b[max_i] = b[max_i], b[k]

        L_k = elementary_lower_triangular_matrix(k, A)
        A = L_k @ A
        b = L_k @ b

    # Обратный ход
    x = numpy.zeros(n)
    x[n - 1] = b[n - 1] / A[n - 1, n - 1]
    for k in range(n - 2, -1, -1):
        x[k] = (b[k] - (A[k, k + 1:] @ x[k + 1:])) / A[k, k]

    return x

# Анализ методов

## 1. Скорость выполнения

In [8]:
def runtime_check(A: numpy.array, b: numpy.array, method: callable) -> tuple[float, numpy.array]:
    start = time.time()
    x = method(A.copy(), b.copy())
    end = time.time() - start

    return end, x

## 2. Погрешность решения

In [9]:
def error_check(x: numpy.array, x_true: numpy.array) -> float:
    return numpy.linalg.norm(x - x_true)

## 3. Устойчивость решения

In [10]:
def stability_check(A: numpy.array, b: numpy.array, x: numpy.array, n: int, method: callable, epsilon: float) -> float:
    delta_b = numpy.random.rand(n)
    delta_b = delta_b / numpy.linalg.norm(delta_b) * epsilon

    delta_x = method(A.copy(), b + delta_b)

    solution_relative_perturbation = numpy.linalg.norm(delta_x - x) / numpy.linalg.norm(x)
    return solution_relative_perturbation

## Построение графика

In [11]:
def show_graphic(sizes: list[int], by_matrix: list[float], by_column: list[float], by_numpy: list[float], title: str, xaxis_title: str, yaxis_title: str) -> None:
    fig = px.line()

    fig.add_scatter(x=sizes, y=by_matrix, name="Гаусс с выбором максимума по матрице")
    fig.add_scatter(x=sizes, y=by_column, name="Гаусс с выбором максимума по столбцу")
    fig.add_scatter(x=sizes, y=by_numpy, name="Встроенный Гаусс из numpy")

    fig.update_layout(
        title = title,
        xaxis_title = xaxis_title,
        yaxis_title = yaxis_title
    )

    fig.show()

## Вывод

In [12]:
def analysis(max_n: int, epsilon: float) -> None:
    sizes = [i for i in range(1, max_n + 1)]

    times_matrix = []
    times_column = []
    times_numpy = []

    errors_matrix = []
    errors_column = []
    errors_numpy = []

    stability_matrix = []
    stability_column = []
    stability_numpy = []

    for n in sizes:
        # Генерация случайной системы Ax = b
        A = numpy.random.rand(n, n)
        x_true = numpy.random.rand(n)
        b = A @ x_true
        V = numpy.vander(numpy.linspace(0, 1, n))
        
        # Гаусс с выбором максимума по матрице
        t, x = runtime_check(A, b, gaussian_method_with_maximum_element_by_matrix)
        times_matrix.append(t)
        errors_matrix.append(error_check(x, x_true))
        stability_matrix.append(stability_check(V, b, x, n, gaussian_method_with_maximum_element_by_matrix, epsilon))
        
        # Гаусс с выбором максимума по столбцу
        t, x = runtime_check(A, b, gaussian_method_with_maximum_element_by_column)
        times_column.append(t)
        errors_column.append(error_check(x, x_true))
        stability_column.append(stability_check(V, b, x, n, gaussian_method_with_maximum_element_by_column, epsilon))

        # Встроенный Гаусс из numpy
        t, x = runtime_check(A, b, numpy.linalg.solve)
        times_numpy.append(t)
        errors_numpy.append(error_check(x, x_true))
        stability_numpy.append(stability_check(V, b, x, n, numpy.linalg.solve, epsilon))

    show_graphic(
        sizes=sizes, 
        by_matrix=times_matrix, 
        by_column=times_column, 
        by_numpy=times_numpy,
        title="График скорости вычисления", 
        xaxis_title="Размер матрицы n", 
        yaxis_title="Время выполнения (сек)"
    )
    show_graphic(
        sizes=sizes, 
        by_matrix=errors_matrix, 
        by_column=errors_column, 
        by_numpy=errors_numpy,
        title="График точности методов", 
        xaxis_title="Размер матрицы n", 
        yaxis_title="Погрешность ||x - x_true||"
    )
    show_graphic(
        sizes=sizes, 
        by_matrix=stability_matrix, 
        by_column=stability_column, 
        by_numpy=stability_numpy,
        title=f"График устойчивости решения при возмущении вектора b на {epsilon * 100}% с матрицей Вандермонда",
        xaxis_title="Размер матрицы n",
        yaxis_title="Относительное возмущение решения (в процентах)"
    )

In [13]:
analysis(500, 1e-3)

/home/tonitrum/Projects/Work/ComputationalMath/.venv/lib/python3.13/site-packages/numpy/linalg/_linalg.py:2792: RuntimeWarning: overflow encountered in dot
  sqnorm = x.dot(x)
